In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from scipy.stats import genextreme

In [2]:
os.chdir("/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/utils")
import dataset_copy

In [ ]:
os.chdir("/work/FAC/FGSE/IDYST/tbeucler/downscaling/HadGEM_driven_COSMO/Present/2003")

filenames = dataset_copy.getfiles()
aout=dataset_copy.new_dataset(filenames,212,243)

aout = aout.drop('lon')
aout = aout.drop('lat')

#On s'intéresse aux données sur la suisse : 
lat_bnd = [-1, 1]
lon_bnd = [-3,0.5]
aout=aout.sel(rlon=slice(*lon_bnd), rlat=slice(*lat_bnd))

In [3]:
path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/data"
os.chdir(path)
mars=xr.open_dataset("mars_1999_2010.nc")
mars

<xarray.Dataset>
Dimensions:    (time: 8184, rlat: 101, rlon: 176)
Coordinates:
  * time       (time) datetime64[ns] 1999-03-02 ... 2009-04-01T23:00:00
    year       (time) int32 ...
  * rlon       (rlon) float32 -3.0 -2.98 -2.96 -2.94 ... 0.44 0.46 0.48 0.5
  * rlat       (rlat) float32 -1.0 -0.98 -0.96 -0.94 ... 0.94 0.96 0.98 1.0
Data variables:
    T_2M       (time, rlat, rlon) float32 ...
    RELHUM_2M  (time, rlat, rlon) float32 ...
    TOT_PR     (time, rlat, rlon) float32 ...
    height_2m  float32 ...
Attributes:
    standard_name:  air_temperature
    long_name:      2m temperature
    units:          K
    grid_mapping:   rotated_pole

In [9]:
import xarray as xr
import numpy as np

# Create new dataset with coordinates


<xarray.Dataset>
Dimensions:         (rlon: 176, rlat: 101, rlat_recreated: 101)
Coordinates:
  * rlon            (rlon) float32 -3.0 -2.98 -2.96 -2.94 ... 0.44 0.46 0.48 0.5
  * rlat            (rlat) float32 -1.0 -0.98 -0.96 -0.94 ... 0.94 0.96 0.98 1.0
  * rlat_recreated  (rlat_recreated) float32 -1.0 -0.98 -0.96 ... 0.96 0.98 1.0
Data variables:
    TOT_PR          (rlat_recreated, rlon) float64 0.005022 0.00257 ... 0.003445
    T_2M            (rlat_recreated, rlon) float64 274.9 281.1 ... 279.5 279.4
    RELHUM_2M       (rlat_recreated, rlon) float64 92.2 99.7 ... 89.65 89.49


In [6]:
import xarray as xr
ds=mars
# Assuming your dataset is named 'ds'
# Select the variables of interest
ds_selected = ds[['TOT_PR', 'T_2M', 'RELHUM_2M']]

# Resample the dataset to monthly frequency
ds_monthly = ds_selected.resample(time='1M').max()

# Find the indices of the maximum values along the time dimension
max_indices = ds_selected['TOT_PR'].argmax(dim='time')

# Extract the maximum TOT_PR values and corresponding T_2M and RELHUM_2M values
max_tot_pr = ds_selected['TOT_PR'].isel(time=max_indices)
max_t_2m = ds_selected['T_2M'].isel(time=max_indices)
max_relhum_2m = ds_selected['RELHUM_2M'].isel(time=max_indices)

recreated_ds = xr.Dataset(coords={'rlon': ds['rlon'], 'rlat': ds['rlat']})
recreated_tot_pr = xr.DataArray(np.nan, coords=[ds['rlat'], ds['rlon']], dims=['rlat', 'rlon'])
recreated_t_2m = xr.DataArray(np.nan, coords=[ds['rlat'], ds['rlon']], dims=['rlat', 'rlon'])
recreated_relhum_2m = xr.DataArray(np.nan, coords=[ds['rlat'], ds['rlon']], dims=['rlat', 'rlon'])

recreated_tot_pr.loc[{'rlat': rlat, 'rlon': rlon}] = max_tot_pr
recreated_t_2m.loc[{'rlat': rlat, 'rlon': rlon}] = max_t_2m
recreated_relhum_2m.loc[{'rlat': rlat, 'rlon': rlon}] = max_relhum_2m
#max_monthly = dataset['TOT_PR'].groupby('time.year').max(dim='time')
print(recreated_ds)

IndexError: dimension coordinate 'rlat' conflicts between indexed and indexing objects:
<xarray.DataArray 'rlat' (rlat: 101)>
array([-1.000000e+00, -9.800000e-01, -9.600000e-01, -9.400000e-01,
       -9.200000e-01, -9.000000e-01, -8.800000e-01, -8.600000e-01,
       -8.400000e-01, -8.200000e-01, -8.000000e-01, -7.800000e-01,
       -7.600000e-01, -7.400000e-01, -7.200000e-01, -7.000000e-01,
       -6.800000e-01, -6.600000e-01, -6.400000e-01, -6.200000e-01,
       -6.000000e-01, -5.800000e-01, -5.600000e-01, -5.400000e-01,
       -5.200000e-01, -5.000000e-01, -4.800000e-01, -4.600000e-01,
       -4.400000e-01, -4.200000e-01, -4.000000e-01, -3.800000e-01,
       -3.600000e-01, -3.400000e-01, -3.200000e-01, -3.000000e-01,
       -2.800000e-01, -2.600000e-01, -2.400000e-01, -2.200000e-01,
       -2.000000e-01, -1.800000e-01, -1.600000e-01, -1.400000e-01,
       -1.200000e-01, -1.000000e-01, -8.000000e-02, -6.000000e-02,
       -4.000000e-02, -2.000000e-02,  8.777701e-16,  2.000000e-02,
   

In [18]:
mars_b=mars.groupby('year')
list_year_data=[]
for year, year_data in mars_b:
        weekly_data = year_data.resample(time='1W').max(dim='time') ##j'ai 5 semaines et le max par semaine !! il faut remplacer time par week
        tot_pr=weekly_data['TOT_PR']
        max_pr_week = tot_pr.groupby('time.week').max(dim='time')
        max_pr_week = max_pr_week.assign_coords(year=year)  # Ajouter l'année en tant que coordonnée
       # _with_year = max_pr_week.expand_dims('year'=year)
        list_year_data.append(max_pr_week)
maxima_mars=xr.concat(list_year_data,dim='week')

""" 
tot_pr_weekly = weekly_data['TOT_PR']
    

# Utilisation de la fonction expand_dims pour ajouter la dimension 'year' à data


# Modification des coordonnées pour remplacer 'time' par 'year'
data_with_year = data_with_year.rename({'time': 'year'})

# Mettre à jour les attributs si nécessaire
data_with_year.attrs['long_name'] = 'total precipitation rate'

    ax(dim='time')
        result = xr.Dataset({'max_pr_week': max_pr_weekly})
    
        weekly_data=weekly_data.assign_coords(
        tot_pr_week=weekly_data['TOT_PR']
        max_pr_week=tot_pr_week.groupby('time.week').max()
        print(max_pr_week)
        """

/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/dev_env/lib/python3.9/site-packages/xarray/core/accessor_dt.py:418: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/dev_env/lib/python3.9/site-packages/xarray/core/accessor_dt.py:418: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/dev_env/lib/python3.9/site-packages/xarray/core/accessor_dt.py:418: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/dev_env/lib/python3.9/site-packages/xarray/core/accessor_dt.py:418: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/work/FAC/FGSE/IDYST/tbeucler/do

" \n\n\ntot_pr_weekly = weekly_data['TOT_PR']\n    \n\n# Utilisation de la fonction expand_dims pour ajouter la dimension 'year' à data\n\n\n# Modification des coordonnées pour remplacer 'time' par 'year'\ndata_with_year = data_with_year.rename({'time': 'year'})\n\n# Mettre à jour les attributs si nécessaire\ndata_with_year.attrs['long_name'] = 'total precipitation rate'\n\n    ax(dim='time')\n        result = xr.Dataset({'max_pr_week': max_pr_weekly})\n    \n        weekly_data=weekly_data.assign_coords(\n        tot_pr_week=weekly_data['TOT_PR']\n        max_pr_week=tot_pr_week.groupby('time.week').max()\n        print(max_pr_week)\n        "

In [30]:
os.chdir("/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/utils")
import dataset_copy
dataset_copy.save_dataset(maxima_mars,'maxima_mars')

saving to  ./maxima_mars.nc
finished saving


In [10]:
maxima_mars_bis.max_pr_week

<xarray.DataArray 'max_pr_week' (rlat: 101, rlon: 176)>
array([[0.00159005, 0.00176227, 0.00168974, ..., 0.00076036, 0.00084292,
        0.00072458],
       [0.00175953, 0.00164954, 0.00160479, ..., 0.00071511, 0.00073265,
        0.00072736],
       [0.001747  , 0.001763  , 0.0016477 , ..., 0.00066693, 0.00071984,
        0.00071599],
       ...,
       [0.00020872, 0.00020827, 0.00020591, ..., 0.00035657, 0.00034972,
        0.00036523],
       [0.00022604, 0.0002129 , 0.00020355, ..., 0.00034198, 0.00036707,
        0.00037309],
       [0.00023833, 0.00023169, 0.00022171, ..., 0.00036883, 0.00035485,
        0.00034904]], dtype=float32)
Coordinates:
  * rlon     (rlon) float32 -3.0 -2.98 -2.96 -2.94 -2.92 ... 0.44 0.46 0.48 0.5
  * rlat     (rlat) float32 -1.0 -0.98 -0.96 -0.94 -0.92 ... 0.94 0.96 0.98 1.0
    week     int64 9
Attributes:
    standard_name:  precipitation_rate
    long_name:      total precipitation rate
    units:          kg m-2 s-1
    grid_mapping:   rotated_pole
    coordinates:    lon lat

In [20]:
dataset_with_maximas_over_week(mars)

/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/dev_env/lib/python3.9/site-packages/xarray/core/accessor_dt.py:418: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(


ValueError: all arrays must be same length

In [11]:
import dealing_with_maxima

In [21]:
mars_maxima=dealing_with_maxima.dataset_with_maximas_over_week(mars)

<xarray.Dataset>
Dimensions:      (rlon: 176, rlat: 101, week: 53, time: 8184)
Coordinates:
  * rlon         (rlon) float32 -3.0 -2.98 -2.96 -2.94 ... 0.44 0.46 0.48 0.5
  * rlat         (rlat) float32 -1.0 -0.98 -0.96 -0.94 ... 0.94 0.96 0.98 1.0
  * week         (week) int64 1 2 3 4 5 6 7 8 9 ... 45 46 47 48 49 50 51 52 53
  * time         (time) datetime64[ns] 1999-03-02 ... 2009-04-01T23:00:00
    year         (time) int64 1999 1999 1999 1999 1999 ... 2009 2009 2009 2009
Data variables:
    max_pr_week  (week, rlat, rlon) float32 nan nan nan nan ... nan nan nan nan


/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/dev_env/lib/python3.9/site-packages/xarray/core/accessor_dt.py:418: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(


In [15]:
janv_maxima.max_pr_week

<xarray.DataArray 'max_pr_week' (week: 53, rlat: 101, rlon: 176)>
array([[[1.64325139e-03, 1.66453945e-03, 1.69910886e-03, ...,
         2.80276779e-03, 3.35098081e-03, 3.58483801e-03],
        [1.57934497e-03, 1.66419812e-03, 1.58876250e-03, ...,
         2.31128396e-03, 2.80428748e-03, 2.56213313e-03],
        [1.35605538e-03, 1.57910457e-03, 1.64360157e-03, ...,
         1.92562060e-03, 2.09847232e-03, 1.48227660e-03],
        ...,
        [3.53014818e-03, 4.18593362e-03, 4.09333874e-03, ...,
         8.03411764e-04, 8.89050891e-04, 1.06248166e-03],
        [9.19032434e-04, 1.12142332e-03, 2.09867163e-03, ...,
         9.65210551e-04, 9.60561214e-04, 7.95988948e-04],
        [8.86973226e-04, 7.96405773e-04, 8.40572582e-04, ...,
         8.41797213e-04, 6.51238544e-04, 5.73625788e-04]],

       [[8.13624429e-05, 8.76833074e-05, 1.98154186e-04, ...,
         1.52621098e-04, 1.20864934e-04, 2.38478096e-04],
        [9.35352145e-05, 1.37957788e-04, 2.73269397e-04, ...,
         1.38534917e-04, 1.18751785e-04, 1.64864468e-04],
        [1.54681053e-04, 2.27899145e-04, 3.60021368e-04, ...,
         1.41988523e-04, 1.11556452e-04, 1.12673893e-04],
...
        [1.15835843e-04, 1.20471304e-04, 1.89147308e-04, ...,
         1.39095282e-04, 2.81768764e-04, 3.63726605e-04],
        [1.30150613e-04, 1.12939917e-04, 1.07803367e-04, ...,
         1.76025744e-04, 2.63418275e-04, 2.72190286e-04],
        [1.57549657e-04, 1.64876532e-04, 1.26204555e-04, ...,
         1.58375697e-04, 1.53080371e-04, 1.26517931e-04]],

       [[1.13286485e-03, 8.04729585e-04, 8.03032075e-04, ...,
         4.00281016e-04, 3.11792683e-04, 2.50581070e-04],
        [1.20860955e-03, 7.34226953e-04, 7.21784367e-04, ...,
         3.61856015e-04, 2.75363505e-04, 2.38350316e-04],
        [1.02522236e-03, 6.54418604e-04, 6.70796668e-04, ...,
         3.53165175e-04, 2.61158682e-04, 2.08543919e-04],
        ...,
        [8.74829304e-04, 6.09905750e-04, 4.77757829e-04, ...,
         3.03804147e-04, 3.10524541e-04, 2.95360864e-04],
        [1.47145521e-03, 1.02933019e-03, 6.78654818e-04, ...,
         3.41543579e-04, 3.35137913e-04, 3.14163277e-04],
        [1.84933376e-03, 1.46525819e-03, 9.88157582e-04, ...,
         3.30482144e-04, 3.20826861e-04, 2.90750788e-04]]], dtype=float32)
Coordinates:
  * rlon        (rlon) float32 -3.0 -2.98 -2.96 -2.94 ... 0.44 0.46 0.48 0.5
  * rlat        (rlat) float32 -1.0 -0.98 -0.96 -0.94 ... 0.94 0.96 0.98 1.0
    lon         (rlat, rlon) float32 5.686 5.715 5.744 ... 10.69 10.72 10.75
    lat         (rlat, rlon) float32 45.92 45.92 45.92 45.92 ... 48.0 48.0 48.0
    height_2m   float32 ...
    height_10m  float32 ...
  * week        (week) int64 1 2 3 4 5 6 7 8 9 10 ... 45 46 47 48 49 50 51 52 53
Attributes:
    standard_name:  precipitation_rate
    long_name:      total precipitation rate
    units:          kg m-2 s-1
    grid_mapping:   rotated_pole

In [3]:
path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/data"
os.chdir(path)
aout_maxima=xr.open_dataset("aout_maxima_suisse.nc")

In [4]:
aout_maxima.max_pr_weekly

<xarray.DataArray 'max_pr_weekly' (week: 6, rlat: 101, rlon: 176)>
[106656 values with dtype=float32]
Coordinates:
  * rlon        (rlon) float32 -3.0 -2.98 -2.96 -2.94 ... 0.44 0.46 0.48 0.5
  * rlat        (rlat) float32 -1.0 -0.98 -0.96 -0.94 ... 0.94 0.96 0.98 1.0
  * week        (week) int32 31 32 33 34 35 36
    height_2m   float32 ...
    height_10m  float32 ...
Attributes:
    standard_name:  precipitation_rate
    long_name:      total precipitation rate
    units:          kg m-2 s-1
    grid_mapping:   rotated_pole

In [5]:

def gev_parameters(dataset):
    """
    returns an xarray with, for each rlon and rlat : the three parameters of the GEV for a dataset that contains 'max_pr_week' variable
    """
    # Dimensions du xarray
    rlon_dim = len(dataset['rlon'])
    rlat_dim = len(dataset['rlat'])
    
    # Résultats
    shape = np.zeros((rlat_dim, rlon_dim))
    loc = np.zeros((rlat_dim, rlon_dim))
    scale = np.zeros((rlat_dim, rlon_dim))
    
    # Boucle for pour calculer les paramètres de la GEV pour chaque point
    for i in range(rlat_dim):
        for j in range(rlon_dim):
            data = dataset['max_pr_weekly'][:, i, j].values
            shape[i, j], loc[i, j], scale[i, j] = genextreme.fit(data)
    
    # Création d'un nouvel xarray pour stocker les résultats
    gev_params_2 = xr.Dataset(
        {
            'shape': (['rlat', 'rlon'], shape),
            'loc': (['rlat', 'rlon'], loc),
            'scale': (['rlat', 'rlon'], scale),
        },
        coords={'rlat': dataset['rlat'], 'rlon': dataset['rlon']}
    )
    return gev_params_2

In [6]:
gev_params_aout_max=gev_parameters(aout_maxima)

In [9]:
os.chdir("/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/utils")
import dataset_copy

dataset_copy.save_dataset(gev_params_aout_max,'gev_params_aout')

saving to  ./gev_params_aout.nc
finished saving


In [2]:
path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/data"
os.chdir(path)
gev_params_aout=xr.open_dataset("gev_params_aout.nc")
gev_params_aout

<xarray.Dataset>
Dimensions:  (rlon: 176, rlat: 101)
Coordinates:
  * rlon     (rlon) float32 -3.0 -2.98 -2.96 -2.94 -2.92 ... 0.44 0.46 0.48 0.5
  * rlat     (rlat) float32 -1.0 -0.98 -0.96 -0.94 -0.92 ... 0.94 0.96 0.98 1.0
Data variables:
    shape    (rlat, rlon) float64 ...
    loc      (rlat, rlon) float64 ...
    scale    (rlat, rlon) float64 ...

In [9]:
gev_params_aout["scale"].values

array([[5.75594775e-06, 6.96212349e-06, 1.68828984e-05, ...,
        1.01198078e-03, 1.13475927e-03, 2.26907009e-03],
       [2.06981854e-05, 2.15242871e-05, 1.84308796e-05, ...,
        2.40673997e-03, 1.73464285e-03, 1.45285143e-03],
       [7.67061023e-05, 2.41945047e-05, 1.63791217e-05, ...,
        2.12336720e-03, 1.79048796e-03, 1.82139292e-03],
       ...,
       [4.24971198e-20, 1.51150453e-20, 1.65421619e-20, ...,
        6.63092566e-05, 5.60742873e-05, 3.18266574e-05],
       [6.75753418e-21, 1.36044285e-19, 2.14938056e-21, ...,
        6.57254952e-05, 5.98022454e-05, 4.86243317e-05],
       [3.69557020e-20, 1.21068782e-19, 1.21904099e-20, ...,
        4.04691164e-05, 3.70518327e-05, 3.15614457e-05]])